In [1]:
import numpy as np
import pickle
data = open("datasets/got.txt","r").read()
#chars = list(set(data))
#pickle.dump(chars,open("chars2.pickle","wb"))
chars = pickle.load(open("Pickles and model Weights/chars2.pickle","rb"))
VOCAB_SIZE = len(chars)
print(len(data),VOCAB_SIZE)

2000193 80


In [2]:
#enumerate
char_to_ix = { ch:ix for ix,ch in enumerate(chars) }
ix_to_char = { ix:ch for ix,ch in enumerate(chars) }
SEQ_LENGTH = 60

In [ ]:
#Prepare Data
X_data = np.zeros([len(data)-SEQ_LENGTH-1,SEQ_LENGTH,1])
Y_data = np.zeros([len(data)-SEQ_LENGTH-1,VOCAB_SIZE])
for i in range(0,len(data)-SEQ_LENGTH-1,1):
    seq_in = data[i:i+SEQ_LENGTH]
    out = data[i+SEQ_LENGTH]
    X_data[i] = [ [char_to_ix[ch]] for ch in seq_in  ]
    Y_data[i,char_to_ix[out]] = 1
print(X_data.shape,Y_data.shape)

In [ ]:
#In previous one i have one-hot encoded input, so shape was (samples,seq_length,vocab_size)
#lets normalize what input
X_data_normalized = X_data/VOCAB_SIZE
X_data_normalized.shape

In [3]:
#lets build model
from keras.models import Sequential
from keras.layers import LSTM, Activation, Dense, Dropout

#Use CuDNNLSTM wirh GPUs for faster performance
def createModel():
    HIDDEN_DIM = 700
    NUM_LAYERS = 2
    DROPOUT_PROBABILITY = 0.3
    model = Sequential()
    for _ in range(NUM_LAYERS - 1):
        model.add(LSTM(HIDDEN_DIM,input_shape=(None,1),return_sequences=True))
        model.add(Dropout(DROPOUT_PROBABILITY))
    model.add(LSTM(HIDDEN_DIM))
    model.add(Dropout(DROPOUT_PROBABILITY))
    model.add(Dense(VOCAB_SIZE,activation='softmax'))
    model.compile(loss="categorical_crossentropy",optimizer='adam',metrics=['accuracy'])
    return model

Using TensorFlow backend.


In [4]:
from keras.callbacks import ModelCheckpoint,EarlyStopping
model_checkpoint = ModelCheckpoint('Pickles and model Weights/simplechar_gen_got2.hdf5',monitor='val_loss',verbose=2,mode='max')

EARLY_STOPPING_PATIENCE = 10
early_stopping = EarlyStopping(monitor='val_loss',patience=EARLY_STOPPING_PATIENCE)

callback_list = [model_checkpoint]

BATCH_SIZE = 512
def trainModel(model):
    EPOCHS=10
    model.fit(X_data_normalized,Y_data,epochs=EPOCHS,batch_size=BATCH_SIZE,validation_split=0.2,callbacks=callback_list,
             verbose=1)
    model.save_weights('Pickles and model Weights/simplechar_gen_got2.hdf5')
    
def loadWeights(model):
    model.load_weights('Pickles and model Weights/simplechar_gen_got2.hdf5')

In [5]:
model = createModel()
loadWeights(model)

In [6]:
def generate_text(model,length):
    random_index =   np.random.randint(0,len(data)-SEQ_LENGTH-1)
    X = np.array([ char_to_ix[ch] for ch in data[random_index:random_index+SEQ_LENGTH]]).reshape([SEQ_LENGTH])
    Y = "".join([ix_to_char[ix] for ix in X ])+"\n\nNow Generated Text\n\n"
    print("Inital Text :\n {}".format(Y))
    output=""
    for i in range(length):
        X_test = X.reshape([1,SEQ_LENGTH,1])/VOCAB_SIZE
        newix = np.argmax(X_test)
        output += ix_to_char[newix]
        X = np.append(X,newix)[1:]
    return output

In [7]:
generate_text(model,150)

Inital Text :
 f a beautiful green lake. Deep Lake had been paid for by the

Now Generated Text




'tUJ‘,w2ve0“blMzc:h7\n{GCo}Bm3EjdF”nQPtUJ‘,w2ve0“blMz“blMzz”nQPtUJ‘,w2ve0“blMzzzzzzzzzzzzzzzzzzmWWkp )(u-.Hxq1?arDZ—L5Ni6c:h7\n{GCo}Bm3EjdF”nQPtUJ‘,w2ve0'

#### Looks like crap because it was trained for only 5 epochs
1. Try One hot encode instead of normalizing input
2. Try larger Network
3. Try with different batch sizes.